### TonEd Notebook
This notebook is 

In [1]:
#import our stuff here
import IPython, numpy as np, scipy as sp, matplotlib.pyplot as plt, matplotlib, sklearn, librosa, cmath, math, nussl
from IPython.display import Audio

In [2]:
#test drive to make sure basic stuff works
sr = 44100.
full, sr = librosa.load("samples/EmilyOtt_Full.wav", sr)
Audio(full, rate=sr)